# RG374 FACS analysis IFNAR fl LysMcre CpG

In [1]:
options(warn=-1)

In [2]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(

            library(stats), 
            library(emmeans), 
            library(outliers), # Grubbs outlier detection 
            
            # Data 
            library(tidyverse), 
            library(data.table), 
            library(reactable), 

            # Plotting 
            library(ComplexHeatmap), 
            library(patchwork), 
            library(cowplot), 
            library(ggrepel)

        )
    )
)

In [3]:
random_seed <- 42
set.seed(random_seed)

In [4]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [5]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Parameter 

In [6]:
color$sample_group<- c("D0 +/+"="#66c2a5", "D0 cre/+"="#00634A", "D3 +/+"="#cd34b5", "D3 cre/+"="#FFAC1E", "D6 +/+"="#cd34b5", "D6 cre/+"="#FFAC1E")

In [7]:
contrast_1 <- c(
    
    "(D0 +/+) - (D3 +/+)", 
    "(D0 +/+) - (D6 +/+)", 
    
    "(D0 cre/+) - (D3 cre/+)", 
    "(D0 cre/+) - (D6 cre/+)", 
    
    "(D0 +/+) - (D0 cre/+)", 
    "(D3 +/+) - (D3 cre/+)", 
    "(D6 +/+) - (D6 cre/+)"
    
)

In [8]:
contrast_2 <- c(
    
    "(I +/+) - (II +/+)", 
    "(I +/+) - (II +/+)", 
    "(I +/+) - (III +/+)", 
    "(I +/+) - (IV +/+)", 
    
    "(I cre/+) - (II cre/+)", 
    "(I cre/+) - (II cre/+)", 
    "(I cre/+) - (III cre/+)", 
    "(I cre/+) - (IV cre/+)", 

    "(I +/+) - (I cre/+)", 
    "(II +/+) - (II cre/+)", 
    "(III +/+) - (III cre/+)", 
    "(IV +/+) - (IV cre/+)"

)

# Functions 

In [9]:
# Sidak test 
sidak <- function(mat, formula_aov, formula_sidak, verbose=FALSE) {
    
    # Perform two-way ANOVA
    anova_result <- stats::aov(formula_aov, data=mat)

    # Post-hoc pairwise comparisons with Sidak correction
    pairwise_comparisons <- emmeans::emmeans(anova_result, formula_sidak, adjust="sidak")
    
    # View the results
    if(verbose) {

        print(pairwise_comparisons$emmeans)  # Estimated marginal means
        print(pairwise_comparisons$contrasts)  # Pairwise comparisons with Sidak adjustment

    }

    # Optional: Display pairwise comparisons in a readable format
    res <- summary(pairwise_comparisons$contrasts)
    
    return(res)
    
}

In [10]:
# Make labels 
scale_label <- function(x, y) {
    
    max_y <- max(x[[y]])
    float_y <- attr(regexpr("(?<=\\.)0+", max_y, perl = TRUE), "match.length")
    
    if(max_y>=1) {
    
        x <- x
        ylab="10^6~x~cells/spleen"
        
        return(list(mat=x, ylab=ylab))

    } else if (float_y==-1) {
        
        x[[y]] <- x[[y]]*10
        ylab <- paste0("10^5~x~cells/spleen")

        return(list(mat=x, ylab=ylab)) 
    
    } else {

        exp_scale <- 1+float_y

        x[[y]] <- x[[y]]*10^exp_scale
        ylab <- paste0("10^", 6-exp_scale, "~x~cells/spleen")

        return(list(mat=x, ylab=ylab))

    }
        
}

In [11]:
# Bar plot
bar_plot <- function(mat, x, y, xlab="", ylab="", ggtitle="", scale_y=FALSE, parse_y=FALSE, expand_y=NULL, position_bar="stack", position_jitter="jitter", position="default") {
    
    if(scale_y) {
        
        res <- scale_label(mat, deparse(substitute(y)))
        mat <- res[[1]]
        ylab <- res[[2]]

    }

    if(position=="dodge") {

        position_bar <- position_dodge(width=0.8)
        position_jitter <- position_dodge(width=0.8)
        
    }
    
    p <- ggplot(mat, aes(x={{x}}, y={{y}}, color=sample_group, fill=sample_group)) + 
    
        geom_bar(stat="summary", width=0.8, color="black", size=0.1, fun=mean, position=position_bar) + 
        
        stat_summary(fun.data = function(x) {
            m <- mean(x)
            sem <- sd(x, na.rm=TRUE) / sqrt(sum(!is.na(x)))
            ymin <- mean(x)
            ymax <- m + sem
            data.frame(y = m, ymin = ymin, ymax = ymax)
        }, geom="errorbar", width=0.25, color="black") +
    
        geom_jitter(shape=21, stroke=0.1, size=1.0, show.legend=FALSE, color="black", position=position_jitter) + 
        xlab(xlab) + ylab(ylab) + ggtitle(ggtitle) + 
        scale_color_manual(values=color$sample_group) + 
        scale_fill_manual(values=color$sample_group) + 
        theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
        guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))
    
    if(parse_y) {
        
        p <- p + ylab(parse(text=ylab))
        
    }


    if(!is.null(expand_y)) {

         p <- p + ylim(0, (max(ggplot_build(p)$data[[1]]$y) + expand_y))
    }
    
    return(p)
    
}

# Import data 

In [12]:
sheet_names <- openxlsx::getSheetNames("data/RG374_ifnar_fl_lysmcre_cpg/facs/spleen//RG374_facs_spleen_data.xlsx")

In [13]:
data <- lapply(sheet_names, function(x) openxlsx::read.xlsx("data/RG374_ifnar_fl_lysmcre_cpg/facs/spleen//RG374_facs_spleen_data.xlsx", sheet=x))
names(data) <- sheet_names

## Spleen weight 

In [14]:
# Prepare data 
mat <- data[["spleen_overview"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample, -viable_cells) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>%
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(sample_group=paste(time_point, genotype)) %>% 
    dplyr::mutate(sample_group=factor(sample_group, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6")))

In [15]:
mat <- split(mat, f=mat$sample_group)

In [16]:
mat <- lapply(mat, function(x) {

    if(length(unique(x$spleen_weight_mg))>=3) {
        
        x <- x %>% dplyr::mutate(
            
            p_value_grubbs=grubbs.test(spleen_weight_mg)$p.value, 
            grubbs_alt=grubbs.test(spleen_weight_mg)$alternative,
            grubbs_res=case_when(
                
                p_value_grubbs < 0.05 & grepl("highest", grubbs_alt) & spleen_weight_mg == max(spleen_weight_mg) ~ TRUE,
                p_value_grubbs < 0.05 & grepl("lowest",  grubbs_alt) & spleen_weight_mg == min(spleen_weight_mg) ~ TRUE,
                TRUE ~ FALSE
            
            )
        )
    } else {

        x <- x %>% dplyr::mutate(

            p_value_grubbs=NA, 
            grubbs_alt=NA,
            grubbs_res=FALSE
            
        )
        
    }

    return(x)
}
             )

In [17]:
mat <- do.call("rbind", mat)
table(mat$grubbs_res)


FALSE  TRUE 
   30     1 

In [18]:
# Remove outlier
mat <- mat[!mat$grubbs_res, ]

In [19]:
# Sidak test
stat <- sidak(mat, formula(spleen_weight_mg ~ time_point * genotype), formula(pairwise ~ time_point * genotype), verbose=FALSE)

In [20]:
stat[stat$contrast %in% contrast_1 & stat$p.value<=0.05, ] %>% dplyr::mutate(p.value=round(p.value, digits=3))

,contrast,estimate,SE,df,t.ratio,p.value
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,(D0 +/+) - (D3 +/+),-70.75000,16.98748,24,-4.164833,0.005
2,(D0 +/+) - (D6 +/+),-104.08333,16.98748,24,-6.127063,0.000
12,(D6 +/+) - (D6 cre/+),72.63333,15.93567,24,4.557910,0.002


In [21]:
bar_plot_1 <- bar_plot(mat, sample_group, spleen_weight_mg, ylab="Spleen weight [mg]", ggtitle="Spleen Weight") + theme(legend.position="none") + theme_global_set(4)
bar_plot_1 <- egg::set_panel_size(bar_plot_1, width=unit(1.8, "cm"), height=unit(2.0, "cm"))

In [22]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_spleen_weight.pdf", width=1.5, height=1.9)

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2

## Viable cells 

In [23]:
mat <- data[["spleen_overview"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample, -spleen_weight_mg) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(sample_group=paste(time_point, genotype)) %>% 
    dplyr::mutate(sample_group=factor(sample_group, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(viable_cells=viable_cells/10^4)

In [24]:
mat <- split(mat, f=mat$sample_group)

In [25]:
mat <- lapply(mat, function(x) {

    if(length(unique(x$viable_cells))>=3) {
        
        x <- x %>% dplyr::mutate(
            
            p_value_grubbs=grubbs.test(viable_cells)$p.value, 
            grubbs_alt=grubbs.test(viable_cells)$alternative,
            grubbs_res=case_when(
                
                p_value_grubbs < 0.05 & grepl("highest", grubbs_alt) & viable_cells == max(viable_cells) ~ TRUE,
                p_value_grubbs < 0.05 & grepl("lowest",  grubbs_alt) & viable_cells == min(viable_cells) ~ TRUE,
                TRUE ~ FALSE
            
            )
        )
    } else {

        x <- x %>% dplyr::mutate(

            p_value_grubbs=NA, 
            grubbs_alt=NA,
            grubbs_res=FALSE
            
        )
        
    }

    return(x)
}
             )

In [26]:
mat <- do.call("rbind", mat)
table(mat$grubbs_res)


FALSE  TRUE 
   29     2 

In [27]:
# Remove outlier
mat <- mat[!mat$grubbs_res, ]

In [28]:
# Sidak test
stat <- sidak(mat, formula(viable_cells ~ time_point * genotype), formula(pairwise ~ time_point * genotype), verbose=FALSE)

In [29]:
stat[stat$contrast %in% contrast_1 & stat$p.value<=0.05, ] %>% dplyr::mutate(p.value=round(p.value, digits=3))

,contrast,estimate,SE,df,t.ratio,p.value
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,(D0 +/+) - (D6 +/+),6.201167,1.808863,23,3.428212,0.034


In [30]:
bar_plot_1 <- bar_plot(mat, sample_group, viable_cells, ylab="10^4~viable~cells/spleen", parse_y=TRUE, ggtitle="Spleen viable cells") + theme(legend.position="none") + theme_global_set(4)
bar_plot_1 <- egg::set_panel_size(bar_plot_1, width=unit(1.8, "cm"), height=unit(2.0, "cm"))

In [31]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_facs_viable_cells.pdf", width=1.5, height=1.9)

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2

## CD44 SSC gating 

In [32]:
mat <- data[["cd44_ssc_gating"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(sample_group=paste(time_point, genotype)) %>% 
    dplyr::mutate(sample_group=factor(sample_group, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    reshape2::melt(., id.vars=c("sample_group", "genotype", "time_point"), variable="celltype", value.name=c("ratio")) %>% 
    dplyr::mutate(sample_group=factor(paste(celltype, sample_group))) %>% dplyr::mutate(sample_group=factor(sample_group, levels=unique(sample_group)))

In [33]:
mat <- split(mat, f=mat$sample_group)

In [34]:
mat <- lapply(mat, function(x) {

    if(length(unique(x$ratio))>=3) {
        
        x <- x %>% dplyr::mutate(
            
            p_value_grubbs=grubbs.test(ratio)$p.value, 
            grubbs_alt=grubbs.test(ratio)$alternative,
            grubbs_res=case_when(
                
                p_value_grubbs < 0.05 & grepl("highest", grubbs_alt) & ratio == max(ratio) ~ TRUE,
                p_value_grubbs < 0.05 & grepl("lowest",  grubbs_alt) & ratio == min(ratio) ~ TRUE,
                TRUE ~ FALSE
            
            )
        )
    } else {

        x <- x %>% dplyr::mutate(

            p_value_grubbs=NA, 
            grubbs_alt=NA,
            grubbs_res=FALSE
            
        )
        
    }

    return(x)
}
             )

In [35]:
mat <- do.call("rbind", mat)
table(mat$grubbs_res)


FALSE  TRUE 
  114    10 

In [36]:
# Remove outlier
mat <- mat[!mat$grubbs_res, ]

In [37]:
# Sidak test
stat <- lapply(levels(mat$time_point), function(i) {sidak(mat[mat$time_point==i, ], formula(ratio ~ celltype * genotype), formula(pairwise ~ celltype * genotype), verbose=FALSE) %>% dplyr::mutate(time_point=i)})
stat <- do.call("rbind", stat)

In [46]:
bar_plot_1 <- bar_plot(mat %>% dplyr::mutate(group=sample_group, sample_group=paste(time_point, genotype)), group, ratio, ylab="10^6~viable~cells/spleen", parse_y=TRUE, position="dodge", ggtitle="Spleen CD44 SSC") + facet_grid(~time_point, scales="free", space="free") + theme(legend.position="none") + theme_global_set(4)
bar_plot_1 <- egg::set_panel_size(bar_plot_1, width=unit(1.8, "cm"), height=unit(2.0, "cm"))

In [47]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_facs_ssc_gating.pdf", width=3*1.5, height=1.9)

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2

# Relative

In [ ]:
mat <- data[["relative"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(sample_group=paste(time_point, genotype)) %>% 
    dplyr::mutate(sample_group=factor(sample_group, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    reshape2::melt(., id.vars=c("sample_group", "genotype", "time_point"), variable="celltype", value.name=c("ratio"))  

In [ ]:
mat <- split(mat, f=paste(mat$sample_group, mat$celltype))

In [ ]:
mat <- lapply(mat, function(x) {

    if(length(unique(x$ratio))>=3) {
        
        x <- x %>% dplyr::mutate(
            
            p_value_grubbs=grubbs.test(ratio)$p.value, 
            grubbs_alt=grubbs.test(ratio)$alternative,
            grubbs_res=case_when(
                
                p_value_grubbs < 0.05 & grepl("highest", grubbs_alt) & ratio == max(ratio) ~ TRUE,
                p_value_grubbs < 0.05 & grepl("lowest",  grubbs_alt) & ratio == min(ratio) ~ TRUE,
                TRUE ~ FALSE
            
            )
        )
    } else {

        x <- x %>% dplyr::mutate(

            p_value_grubbs=NA, 
            grubbs_alt=NA,
            grubbs_res=FALSE
            
        )
        
    }

    return(x)
}
             )

In [ ]:
mat <- do.call("rbind", mat)
table(mat$grubbs_res)

In [ ]:
# Remove outlier
mat <- mat[!mat$grubbs_res, ]

In [ ]:
# Sidak test
stat <- lapply(levels(mat$celltype), function(i) {sidak(mat[mat$celltype==i, ], formula(ratio ~ time_point * genotype), formula(pairwise ~ time_point * genotype), verbose=FALSE) %>% dplyr::mutate(celltype=i)})
stat <- do.call("rbind", stat)

In [ ]:
stat[stat$contrast %in% contrast_1 & stat$p.value<=0.05, ] %>% dplyr::mutate(p.value=round(p.value, digits=3))

In [ ]:
bar_plot_1 <- lapply(split(mat, f=mat$celltype), function(x) bar_plot(x, sample_group, ratio, ylab="Ratio [%]", ggtitle=x$celltype[1]) + theme(legend.position="none") + theme_global_set(4))
bar_plot_1 <- lapply(bar_plot_1, function(p) egg::set_panel_size(p, width=unit(1.8, "cm"), height=unit(2.0, "cm")))
bar_plot_1 <- do.call(gridExtra::arrangeGrob, c(bar_plot_1, ncol=7, nrow=ceiling(length(bar_plot_1)/7)))

In [ ]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_facs_relative.pdf", width=7*1.5, height=1.9*ceiling(length(bar_plot_1)/7))

grid::grid.draw(bar_plot_1)

dev.off()

## Absolut 

In [ ]:
mat <- data[["absolut"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(sample_group=paste(time_point, genotype)) %>% 
    dplyr::mutate(sample_group=factor(sample_group, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    reshape2::melt(., id.vars=c("sample_group", "genotype", "time_point"), variable="celltype", value.name=c("absolut"))  

In [ ]:
mat <- split(mat, f=paste(mat$sample_group, mat$celltype))

In [ ]:
mat <- lapply(mat, function(x) {

    if(length(unique(x$absolut))>=3) {
        
        x <- x %>% dplyr::mutate(
            
            p_value_grubbs=grubbs.test(absolut)$p.value, 
            grubbs_alt=grubbs.test(absolut)$alternative,
            grubbs_res=case_when(
                
                p_value_grubbs < 0.05 & grepl("highest", grubbs_alt) & absolut == max(absolut) ~ TRUE,
                p_value_grubbs < 0.05 & grepl("lowest",  grubbs_alt) & absolut == min(absolut) ~ TRUE,
                TRUE ~ FALSE
            
            )
        )
    } else {

        x <- x %>% dplyr::mutate(

            p_value_grubbs=NA, 
            grubbs_alt=NA,
            grubbs_res=FALSE
            
        )
        
    }

    return(x)
}
             )

In [ ]:
mat <- do.call("rbind", mat)
table(mat$grubbs_res)

In [ ]:
# Remove outlier
mat <- mat[!mat$grubbs_res, ]

In [ ]:
# Sidak test
stat <- lapply(levels(mat$celltype), function(i) {sidak(mat[mat$celltype==i, ], formula(absolut ~ time_point * genotype), formula(pairwise ~ time_point * genotype), verbose=FALSE) %>% dplyr::mutate(celltype=i)})
stat <- do.call("rbind", stat)

In [ ]:
stat[stat$contrast %in% contrast_1 & stat$p.value<=0.05, ] %>% dplyr::mutate(p.value=round(p.value, digits=3))

In [ ]:
bar_plot_1 <- lapply(split(mat, f=mat$celltype), function(x) bar_plot(x, sample_group, absolut, ylab="Absolut", ggtitle=x$celltype[1]) + theme(legend.position="none") + theme_global_set(4))
bar_plot_1 <- lapply(bar_plot_1, function(p) egg::set_panel_size(p, width=unit(1.8, "cm"), height=unit(2.0, "cm")))
bar_plot_1 <- do.call(gridExtra::arrangeGrob, c(bar_plot_1, ncol=7, nrow=ceiling(length(bar_plot_1)/7)))

In [ ]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_facs_absolut.pdf", width=7*1.5, height=1.9*ceiling(length(bar_plot_1)/7))

grid::grid.draw(bar_plot_1)

dev.off()